# Debug de Componentes do Agente Multi-Camada

Este notebook permite testar individualmente cada componente da arquitetura antes de rodar o orquestrador completo.
Isso é útil para validar conexões Spark, acesso ao catálogo de dados, geração de contexto e execução de LLM isoladamente.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.getcwd())

## 1. Teste do Data Engine (Spark Connection)

In [ ]:
from app.data_engine import DataEngine

print("Inicializando DataEngine...")
data_engine = DataEngine()
spark = data_engine.initialize_session()

print(f"Spark Session: {spark}")

# Teste simples de query
try:
    print("Testando query simples 'SELECT 1'...")
    spark.sql("SELECT 1").show()
except Exception as e:
    print(f"[ERRO] Falha no teste Spark: {e}")

## 2. Teste do Context Manager (Schema Injection)

In [ ]:
from app.context_manager import ContextManager

context_mgr = ContextManager(spark)

# Teste com o papel 'logistics' (deve pegar olist_sales e olist_logistics)
role = "logistics"
print(f"Gerando contexto para role: {role}...")

schema_context = context_mgr.get_schema_context(role)

if schema_context:
    print("\n[SUCESSO] Contexto gerado:")
    print(schema_context[:500] + "... (truncado)")
else:
    print("[AVISO] Contexto retornou vazio. Verifique se as tabelas existem no catálogo 'olist_dataset'.")

## 3. Teste do Agente 'Logistics' Isolado (LLM + Tool)

In [ ]:
from app.agents import Agent
from app.tools import SparkSQLTool

# Inicializa Tool
tool = SparkSQLTool(spark)

# Inicializa Agente de Logística
logistics_agent = Agent("LogisticsDebug", "logistics", context_mgr, tool)

print("Executando Agente de Logística...")
# Pergunta simples para testar a ponte LLM -> SQL
response = logistics_agent.run("Liste 5 pedidos da tabela olist_sales.orders (ou similar) para verificar se você consegue ler os dados.")

print("\n=== Resposta Final ===")
print(response)

## 4. Teste do Agente 'Finance' Isolado

In [ ]:
finance_agent = Agent("FinanceDebug", "finance", context_mgr, tool)

print("Executando Agente de Finanças...")
response_fin = finance_agent.run("Qual o total de pagamentos registrados na tabela de pagamentos?")

print("\n=== Resposta Final ===")
print(response_fin)